<a href="https://colab.research.google.com/github/Ananya-AJ/CMPE255-SafeDose/blob/main/Visualisations%20for%20frontend/Dawn_Visualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd.set_option('display.max_columns',None)

In [9]:
dawn_drug = pd.read_csv('drug_data.csv')
dawn_drug.head()
dawn_drug.replace({-7:0,-8:0,-9:0},inplace=True)

Visualizations

##Total cases in each metro and individual case type count in that metro

In [16]:

df_grouped = dawn_drug.groupby(['METRO', 'CASETYPE']).agg(
    count_col=pd.NamedAgg(column='CASETYPE', aggfunc="count")
)
df_grouped["USA_2011"] = "USA_2011"
df_grouped = df_grouped.reset_index()
print(df_grouped)

     METRO  CASETYPE  count_col  USA_2011
0        1         1        534  USA_2011
1        1         2       2901  USA_2011
2        1         3       1604  USA_2011
3        1         4      10089  USA_2011
4        1         5       1710  USA_2011
..     ...       ...        ...       ...
107     14         4      12905  USA_2011
108     14         5       2345  USA_2011
109     14         6         85  USA_2011
110     14         7        601  USA_2011
111     14         8      10347  USA_2011

[112 rows x 4 columns]


In [17]:
fig = px.sunburst(df_grouped,
                  path=["USA_2011", "METRO", "CASETYPE"],
                  values='count_col',
                  title="CASETYPE distribution(counts of cases with this casetype) in each metro",
                  width=750, height=750)
fig.show()

US map highlighted with metros that have drug cases

In [25]:
df_grouped_metro = dawn_drug.groupby(['METRO', 'ALLABUSE']).agg(
    count_col=pd.NamedAgg(column='ALLABUSE', aggfunc="count")
)

In [26]:
df_grouped_metro.head()

count_col
METRO ALLABUSE           
1     0             11021
      1             12806
2     0             10745
      1             27415
3     0              8645

In [36]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Trying to plot df_grouped_metro on the map. I guess we have to have metro mapped to places to show the data according to region?

In [35]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px

fig = px.choropleth(df_grouped_metro, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'Abuse':'METRO'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: ignored

Which drug/drugs are predominant in each type of case (suicide, overmedication, etc) 

In [18]:
#casetype and drugids dataframe 
casetype_drug_df = dawn_drug[['CASETYPE','DRUGID_1','DRUGID_2','DRUGID_3']]
casetype_drug_df.sort_index(axis=1,inplace=True)
casetype_drug_df.head()

,CASETYPE,DRUGID_1,DRUGID_2,DRUGID_3
0,8,865,1255,0
1,4,2077,0,0
2,4,2313,0,0
3,4,234,0,0
4,8,865,1253,1254


for each casetype, drugid's most occuring drugs for that case with counts in below o/p

In [19]:
# drugid_1_mode = casetype_drug_df.DRUGID_1.mode()
# drugid_2_mode = casetype_drug_df.value_counts()[1]
# drugid_3_mode = casetype_drug_df.DRUGID_3.value_counts()[1]

group_casetypes_df = casetype_drug_df.groupby('CASETYPE').aggregate(
            ({'DRUGID_1':['max','count'],'DRUGID_2':['max','count'],'DRUGID_3':['max','count']})
)

group_casetypes_df


DRUGID_1        DRUGID_2        DRUGID_3       
              max  count      max  count      max  count
CASETYPE                                                
1            2558   7872     2558   7872     2558   7872
2            2545  13529     2545  13529     2558  13529
3             865   7421        0   7421        0   7421
4            2558  85777     2558  85777     2558  85777
5            2558  16810     2558  16810     2558  16810
6            2557    768     2558    768     2524    768
7            2558   3125     2557   3125     2442   3125
8            2558  83648     2558  83648     2558  83648

Gender wise casetypes

In [20]:
df_grouped_gender = dawn_drug.groupby(['SEX', 'CASETYPE']).agg(
    count_col=pd.NamedAgg(column='CASETYPE', aggfunc="count")
)
df_grouped_gender= df_grouped_gender.reset_index()
print(df_grouped_gender)

    SEX  CASETYPE  count_col
0     0         1          4
1     0         2          2
2     0         3          4
3     0         4         17
4     0         5          6
5     0         6          1
6     0         7          1
7     0         8         34
8     1         1       3216
9     1         2       9558
10    1         3       4007
11    1         4      33245
12    1         5       7127
13    1         6        263
14    1         7       1548
15    1         8      54753
16    2         1       4652
17    2         2       3969
18    2         3       3410
19    2         4      52515
20    2         5       9677
21    2         6        504
22    2         7       1576
23    2         8      28861


In [21]:
fig = px.sunburst(df_grouped_gender,
                  path=["SEX", "CASETYPE"],
                  values='count_col',
                  title="CASETYPE distribution(counts of cases with this casetype) in each gender",
                  width=750, height=750)
fig.show()

Race - casetypes (to see if there is any pattern between the casetypes and race)

In [22]:
df_grouped_race = dawn_drug.groupby(['RACE', 'CASETYPE']).agg(
    count_col=pd.NamedAgg(column='CASETYPE', aggfunc="count")
)
#df_grouped["USA_2011"] = "USA_2011"
df_grouped_race= df_grouped_race.reset_index()
print(df_grouped_race)

    RACE  CASETYPE  count_col
0      0         1       1078
1      0         2       1330
2      0         3       1481
3      0         4      15416
4      0         5       2419
5      0         6        113
6      0         7        515
7      0         8       9007
8      1         1       4205
9      1         2       6388
10     1         3       3303
11     1         4      43317
12     1         5       9746
13     1         6        385
14     1         7       1546
15     1         8      34936
16     2         1       1335
17     2         2       4263
18     2         3        843
19     2         4      13728
20     2         5       2281
21     2         6        167
22     2         7        466
23     2         8      26004
24     3         1       1025
25     3         2       1464
26     3         3       1451
27     3         4      10506
28     3         5       1898
29     3         6         87
30     3         7        502
31     3         8      12124
32     4  

In [23]:
fig = px.sunburst(df_grouped_race,
                  path=["RACE", "CASETYPE"],
                  values='count_col',
                  title="CASETYPE distribution(counts of cases with this casetype) in each race",
                  width=750, height=750)
fig.show()